In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'urban_centers', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by Regional Geography
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
df.rename(columns={'name': 'Regional Growth Center'}, inplace=True)
df['Regional Growth Center'] = df['Regional Growth Center'].fillna('Not in RGC')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,442,"5,366","8,225","40,832"
Bellevue,"1,533","8,575","15,850","64,946"
Bothell Canyon Park,"2,487","15,810","29,908","124,373"
Bremerton,214,"1,092","2,201","28,077"
Burien,273,"1,376","11,599","70,061"
Everett,83,"10,910","19,345","69,714"
Federal Way,101,"2,473","2,686","23,144"
Issaquah,"1,510","5,437","36,754","174,384"
Kent,"1,226","12,736","14,991","62,936"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.8%,9.8%,15.0%,74.4%
Bellevue,1.7%,9.4%,17.4%,71.4%
Bothell Canyon Park,1.4%,9.2%,17.3%,72.1%
Bremerton,0.7%,3.5%,7.0%,88.9%
Burien,0.3%,1.7%,13.9%,84.1%
Everett,0.1%,10.9%,19.3%,69.7%
Federal Way,0.4%,8.7%,9.5%,81.5%
Issaquah,0.7%,2.5%,16.9%,80.0%
Kent,1.3%,13.9%,16.3%,68.5%


#### AM Peak (7 to 8 AM)

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,87,476,"1,388","2,038"
Bellevue,128,526,948,"4,566"
Bothell Canyon Park,0,"1,731","3,792","5,999"
Bremerton,23,116,169,"2,280"
Burien,16,250,849,"4,458"
Everett,0,942,"1,514","4,451"
Federal Way,6,228,110,"1,434"
Issaquah,95,"2,595","5,628","7,876"
Kent,184,"1,471","1,037","3,806"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,2.2%,11.9%,34.8%,51.1%
Bellevue,2.1%,8.5%,15.4%,74.0%
Bothell Canyon Park,0.0%,15.0%,32.9%,52.1%
Bremerton,0.9%,4.5%,6.5%,88.1%
Burien,0.3%,4.5%,15.2%,80.0%
Everett,0.0%,13.6%,21.9%,64.4%
Federal Way,0.3%,12.8%,6.2%,80.6%
Issaquah,0.6%,16.0%,34.8%,48.6%
Kent,2.8%,22.6%,16.0%,58.6%


#### PM Peak (5 to 6 PM)

In [12]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,48,535,591,"3,413"
Bellevue,138,918,"1,473","5,371"
Bothell Canyon Park,408,"2,519","2,846","7,683"
Bremerton,53,107,199,"2,583"
Burien,25,102,"1,688","5,953"
Everett,23,"1,076","1,651","5,849"
Federal Way,8,225,159,"1,694"
Issaquah,241,351,"9,168","9,175"
Kent,211,"1,374","1,145","4,930"


In [13]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,1.0%,11.7%,12.9%,74.4%
Bellevue,1.8%,11.6%,18.6%,68.0%
Bothell Canyon Park,3.0%,18.7%,21.2%,57.1%
Bremerton,1.8%,3.6%,6.8%,87.8%
Burien,0.3%,1.3%,21.7%,76.6%
Everett,0.3%,12.5%,19.2%,68.0%
Federal Way,0.4%,10.8%,7.6%,81.2%
Issaquah,1.3%,1.9%,48.4%,48.5%
Kent,2.8%,17.9%,14.9%,64.4%


# Network Delay
Totals scaled to match network totals. 

In [14]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
Regional Growth Center,
Auburn,240
Bellevue,282
Bothell Canyon Park,"1,192"
Bremerton,44
Burien,127
Everett,253
Federal Way,45
Issaquah,898
Kent,557


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
Regional Growth Center,
Auburn,228
Bellevue,268
Bothell Canyon Park,"1,133"
Bremerton,42
Burien,121
Everett,240
Federal Way,43
Issaquah,854
Kent,530
